# Alternatieve Toets TDY2
Calculationsheet for a gasturbine  

_note_: The cell below are needed to import libraries and set global settings, used in this sheet

In [1]:
%matplotlib notebook
%precision 3
from IPython.display import Latex
from math import nan
import matplotlib.pyplot as plt
from GasTurbine import *

# Decription of the system
## States
The following states are identified:  
0. Air at atmospheric conditions $ P_0 = P_{atm} $, $ T_0 = T_{atm} $ and $ v_0 = v_{atm} $
1. Compressed air $ P_1 >> P_0 $, $ T_1 > T_0 $ and $ v_1 << v_0 $
2. Heated air $ P_2 = P_1 $, $ T_2 >> T_1 $ and $ v_2 > v_1 $
3. Heated air at atmospheric pressure $ P_3 = P_0 = P_{atm} $, $ T_3 < T_2 $

## Transitions
The system transitions are identified as follows:  
0 -> 1 **Isentropic compression** were air is sucked into a compressor.  
1 -> 2 **Isobaric heating** where air is heated in a combustion chamber.  
2 -> 3 **Isentropic expansion** where air is expanded till atmospheric pressure.  
3 -> 0 **Isobaric cooling** A fictive step where the air is cooled to atmorspheric conditions.  

# Know values

In [2]:
P_atm = 1.013 * u.bar
T_atm = (15. * u.degC).to(u.K)
cycle = Cycle()
cycle.c_p = 1005. * u.J / (u.kg * u.K)
cycle.c_v = 716. * u.J / (u.kg * u.K)
cycle.Q_in = 2000 * u.kW

In [3]:
states = States(4)
states[0].P = P_atm
states[0].T = T_atm
states[0].A = 1. * u.m ** 2
states[0].V_flux = 6000. * u.m ** 3 / u.hr
states[1].P = 10. * u.bar
states[1].A = 0.5 * u.m ** 2
states[2].A = 0.1 * u.m ** 2
states[2].P = states[1].P
states[3].A = 1. * u.m ** 2
states[3].P = states[0].P

In [4]:
cycle.transitions = states
cycle.transitions[0].processtype = ProcessType.ISENTROPIC
cycle.transitions[1].processtype = ProcessType.ISOBARIC
cycle.transitions[2].processtype = ProcessType.ISENTROPIC
cycle.transitions[3].processtype = ProcessType.ISOBARIC

# Calculate of system constants
\begin{equation}
    \frac{P_0 \dot{V_0}}{T_0} = \dot{m} R \rightarrow \dot{m} = \frac{P_0 \dot{V_0}}{R T_0}
\end{equation}
\begin{equation}
    R = c_p - c_v
\end{equation}
\begin{equation}
    k = \frac{c_p}{c_p}
\end{equation}

In [5]:
latex(cycle.R)

<IPython.core.display.Latex object>

In [6]:
latex(cycle.k)

<IPython.core.display.Latex object>

In [7]:
cycle.m_flux = mass_flux(states[0].P, states[0].V_flux, cycle.R, states[0].T)
latex(cycle.m_flux)

<IPython.core.display.Latex object>

# Calculate Transitions and States
## State 0
\begin{equation}
    \rho_0 = \frac{P_0}{T_0 R}
    \begin{cases}
        \frac{P_0 v_0}{T_0} = \dot{m} R \\
        \rho_0 = \frac{\dot{m}}{v_0}
    \end{cases}
\end{equation}
\begin{equation}
    v_0 = \frac{\dot{V_0}}{\dot{m}}
\end{equation}
\begin{equation}
    c_0 A_0 \rho_0 = \dot{m} \rightarrow c_0 = \frac{\dot{m}}{\rho_0 A_0}
\end{equation}


In [8]:
states[0].rho = density(states[0].P, cycle.R, states[0].T)
states[0].v = specific_volume(cycle.m_flux, states[0].V_flux)
states[0].c = speed(cycle.m_flux, states[0].rho, states[0].A)
latex(states[0])

<IPython.core.display.Latex object>

## State 1
Where $ \rho_1 $, $ c_1 $ and $ v_1 $ are determined with the previous given formula.
\begin{equation}
    \frac{T_0^k}{P_0^{k-1}} = \frac{T_1^k}{P_1^{k-1}} \rightarrow T_1 = \left(\left(\frac{P_1}{P_0}\right)^{k-1}T_0^k\right)^{\frac{1}{k}}
\end{equation}
\begin{equation}
    \dot{V}_1 = \frac{\rho_0}{\rho_1} \dot{V}_0
\end{equation}
\begin{equation}
    p_0 V_0^k = p_1 V_1^k \rightarrow v_1 = \left(\frac{P_0}{P_1}v_0^k\right)^{\frac{1}{k}}
\end{equation}

In [9]:
states[1].T = isentropische_dT(states[0].T, states[0].P, states[1].P, cycle.k)
states[1].rho = density(states[1].P, cycle.R, states[1].T)
states[1].c = speed(cycle.m_flux, states[1].rho, states[1].A)
states[1].V_flux = V_flux(states[0].rho, states[1].rho, states[0].V_flux)
states[1].v = specific_volume(cycle.m_flux, states[1].V_flux)
latex(states[1])

<IPython.core.display.Latex object>

## Transition 0 to 1
The transition from **state 0** to **state 1** is an isentropic process where work needs to be put into the system and no heat is transfered.
\begin{equation}
    w_{0-1} = \frac{k}{k-1}\left(P_1 v_1 - P_0 v_0\right)
\end{equation}
\begin{equation}
    q_{0-1} = 0.
\end{equation}
\begin{equation}
    \Delta u = q_{0-1} - w_{0-1}
\end{equation}
\begin{equation}
    n = k
\end{equation}
\begin{equation}
    w_{t,0-1} = \frac{-n}{n-1} (P_1 v_1 - P_0 v_0)
\end{equation}
\begin{equation}
    \Delta e_{kin} = \frac{1}{2}(c_1^2 - c_0^2)
\end{equation}
\begin{equation}
    \Delta h_{0-1} = q_{0-1} - w_{t,0-1} - \Delta e_{kin}
\end{equation}

In [10]:
cycle.transitions[0].processtype = ProcessType.ISENTROPIC
cycle.transitions[0].q = 0. * u.kJ / u.kg
cycle.transitions[0].w = isentropisch_work(states[0].P, states[0].v, states[1].P, states[1].v, cycle.k)
cycle.transitions[0].du = delta_u(cycle.transitions[0].q, cycle.transitions[0].w)
cycle.transitions[0].w_t = technical_work(states[0].P, states[0].v, states[1].P, states[1].v, cycle.k)
cycle.transitions[0].de_kin = delta_e_kin(states[0].c, states[1].c)
cycle.transitions[0].dh = delta_h(cycle.transitions[0].q, cycle.transitions[0].w_t, cycle.transitions[0].de_kin)
latex(cycle.transitions[0])

<IPython.core.display.Latex object>

## State 2
State 2 can best be calculated by first determining the heat and work transfer in transition 1 -> 2. Since the amount of heating energy $ Q_{in} $ is known. The transition from **state 1** to **state 2** is an isobaric heating process. Where heat is put into the system which is kept at an constant pressure.
\begin{equation}
    q_{1-2} = \frac{Q_{in}}{\dot{m}}
\end{equation}
\begin{equation}
    w_{1-2} = \frac{q_{1-2}}{\frac{k}{k-1}}
    \begin{cases}
        q_{1-2} = \frac{k}{k-1} P(v_2 - v_1) \\
        w_{1-2} = P(v_2 - v_1)
    \end{cases}
\end{equation}



In [11]:
cycle.transitions[1].processtype = ProcessType.ISENTROPIC
cycle.transitions[1].q = isobaar_heat(Q_in=cycle.Q_in, m_flux=cycle.m_flux)
cycle.transitions[1].w = isobaar_work(k=cycle.k, q=cycle.transitions[1].q)
cycle.transitions[1].du = delta_u(cycle.transitions[1].q, cycle.transitions[1].w)

From here **state 2** can be calculated, where $ \rho_2 $, $ v_2 $, $ c_2 $ and $ \dot{V}_2 $ are calculated using earlier provided formulas.
\begin{equation}
    q_{1-2} = c_p(T_2-T_1) \rightarrow T_2 = \frac{q_{1-2}}{c_p} + T_1
\end{equation}

In [12]:
states[2].T = isobaar_dT(cycle.transitions[1].q, cycle.c_p, states[2].T)
states[2].rho = density(states[2].P, cycle.R, states[2].T)
states[2].c = speed(cycle.m_flux, states[2].rho, states[2].A)
states[2].V_flux = V_flux(states[1].rho, states[2].rho, states[1].V_flux)
states[2].v = specific_volume(cycle.m_flux, states[2].V_flux)
latex(states[2])

<IPython.core.display.Latex object>

## Transition 1 to 2
From here the transition can be calculated further, using earlier given formulas, where 
\begin{equation}
    n = 0
\end{equation}

In [13]:
cycle.transitions[1].w_t = technical_work(states[1].P, states[1].v, states[2].P, states[2].v, cycle.k)
cycle.transitions[1].de_kin = delta_e_kin(states[1].c, states[2].c)
cycle.transitions[1].dh = delta_h(cycle.transitions[1].q, cycle.transitions[1].w_t, cycle.transitions[1].de_kin)
latex(cycle.transitions[1])

<IPython.core.display.Latex object>

## State 3


In [15]:
states[3].T = isentropische_dT(states[2].T, states[2].P, states[3].P, cycle.k)
states[3].rho = density(states[3].P, cycle.R, states[3].T)
states[3].c = speed(cycle.m_flux, states[3].rho, states[2].A)
states[3].V_flux = V_flux(states[2].rho, states[3].rho, states[2].V_flux)
states[3].v = specific_volume(cycle.m_flux, states[3].V_flux)
latex(states[3])

<IPython.core.display.Latex object>

## Transition 2 to 3


In [17]:
cycle.transitions[2].processtype = ProcessType.ISENTROPIC
cycle.transitions[2].q = 0. * u.kJ / u.kg
cycle.transitions[2].w = isentropisch_work(states[2].P, states[2].v, states[3].P, states[3].v, cycle.k)
cycle.transitions[2].du = delta_u(cycle.transitions[2].q, cycle.transitions[2].w)
cycle.transitions[2].w_t = technical_work(states[2].P, states[2].v, states[3].P, states[3].v, cycle.k)
cycle.transitions[2].de_kin = delta_e_kin(states[2].c, states[3].c)
cycle.transitions[2].dh = delta_h(cycle.transitions[2].q, cycle.transitions[2].w_t, cycle.transitions[2].de_kin)
latex(cycle.transitions[2])

<IPython.core.display.Latex object>

## Transition 3 to 0

In [19]:
cycle.transitions[3].processtype = ProcessType.ISOBARIC
cycle.transitions[3].q = isobaar_heat(c_p=cycle.c_p, T_s=states[3].T, T_e=states[0].T)
cycle.transitions[3].w = isobaar_work(R=cycle.R, T_s=states[3].T, T_e=states[0].T)
cycle.transitions[3].du = delta_u(cycle.transitions[3].q, cycle.transitions[3].w)
cycle.transitions[3].w_t = technical_work(states[3].P, states[3].v, states[0].P, states[0].v, cycle.k)
cycle.transitions[3].de_kin = delta_e_kin(states[3].c, states[0].c)
cycle.transitions[3].dh = delta_h(cycle.transitions[3].q, cycle.transitions[3].w_t, cycle.transitions[3].de_kin)
latex(cycle.transitions[3])

<IPython.core.display.Latex object>

In [21]:
sum([t.q for t in cycle.transitions])

<Quantity(765.4021247128735, 'kilojoule / kilogram')>

In [22]:
sum([t.w for t in cycle.transitions])

<Quantity(366.85399916825656, 'kilojoule / kilogram')>